# 3. Prepare Data and Synthetics for Moment Tensor Inversion

Now, we are about to do the inversion. But before that, we need to prepare the data and the synthetics that are required for moment tensor inversion.
we need to create synthetic Green's functions that is required for the inversion. In this notebook we will do:
- **Filter and cut the seismograms**;
- **Calculate and filter synthetic Green's functions**;
- **Prepare input file for moment tensor inversion**. 

We will use wavenumber integration (FK) method for creating the synthetics. There are many softwares, and here we use the software package ***Computer Programs in Seismology ([CPS](http://www.eas.slu.edu/eqc/eqccps.html))*** by R.B Herrmann.


In [1]:
# Import third-party libraries
import os
from pathlib import Path
import pandas as pd
from obspy.core import read, UTCDateTime, Stream

In [2]:
# 3.1 Filter and cut the seismograms

# Prepare the folders/directories
evid = "41423391" #!!! event of interest
event_dir = evid
infile = "%s/datetime.csv"%event_dir # we need the event origin time
station_file = "%s/station.csv"%event_dir

sacdir = "%s/sac"%event_dir # location of pre-processed data
outdir = "%s"%event_dir     # location of filtered/cut/down-sampled data for inversion
    
# Check if data directory exist
P = Path(sacdir)
if P.exists():
    # Read event info and station info into Pandas table
    df = pd.read_csv(infile,parse_dates=True)
    station_df = pd.read_csv("%s"%(station_file),parse_dates=True,dtype={"location":str},na_filter=False)
    
    origin_time = UTCDateTime(df["origin"][0])
    st = Stream()
    for _,row in station_df.iterrows():
        st += read("%s/%s.%s.%s.%s[%s]"%(
            sacdir,row.network,row.station,row.location,row.channel,row.component),format="SAC")
else:
    print("Directory %s does not exist. %s does not have instrument corrected data."%(sacdir,evid))
    

In [3]:
# Set parameters for processing data
#   You may need to change them for different events. 
#   Synthetic Green's functions must have the same filter, 
#   reduction velocity and sampling interval as the data.

# Filter parameters
freqmin = 0.02
freqmax = 0.05
corners = 3

# Desired sampling interval
dt = 1.0

# Reduction velocity in km/sec, 0 sets the reference time to origin time
vred = 0

# time before and after reference time, data will be cut before and after the reference time
time_before = 30
time_after = 200

In [4]:
if vred:
    p = 1/vred
else:
    p = 0

# filter and taper
st.filter("bandpass",freqmin=freqmin,freqmax=freqmax,corners=corners,zerophase=True)
st.taper(max_percentage=0.05)

# Trim and resample the data
for tr in st:
    ## resample the seismograms
    tr.decimate(factor=int(tr.stats.sampling_rate*dt), strict_length=False, no_filter=True)
    tr.resample(1/dt, strict_length=False, no_filter=True)
    tr.stats.sac.t1 = origin_time + p*(tr.stats.sac.dist) # set reference time
    
    ## trim the seismograms
    tr.trim(tr.stats.sac.t1-time_before,tr.stats.sac.t1+time_after,pad=True,fill_value=0)
    tr.data = 100*tr.data # m/s to cm/s
    tr.stats.sac.b = -1*(origin_time - tr.stats.starttime)
    tr.stats.sac.o = 0
    
    ## Save final trace using tdmtpy file name format
    sacout = "%s/%s.%s.dat"%(outdir,tr.id[:-4],tr.id[-1])
    tr.write(sacout,format="SAC")

In [5]:
# 3.2 Calculate the synthetic Green's functions
#   The calculation requires two input files, a velocity model file and a distance file. 

model  = "gil7" # that means the model file is "gil7.d"
depths = sorted([10,20,30,40,50]) # compute GF at 10, 20, ..., km
npts   = int(256) # number of points in the time series, must be a power of 2
t0     = int(0)   # used to define the first sample point, t0 + distance_in_km/vred

# Location to store synthetic Green's functions
green_dir = "%s/%s"%(event_dir,model)
Path(green_dir).mkdir(parents=True,exist_ok=True)
    
for depth in depths:
    # Create distance file
    dfile = ("{dist:.0f} {dt:.2f} {npts:d} {t0:d} {vred:.1f}\n")
    dfile_out = "%s/dfile"%event_dir
    with open(dfile_out,"w") as f:
        for _,row in station_df.iterrows():
            f.write(dfile.format(dist=row.distance,dt=dt,npts=npts,t0=t0,vred=vred))

    # Generate the synthetics
    os.system("hprep96 -M %s.d -d %s -HS %.4f -HR 0 -EQEX"%(model,dfile_out,depth))
    os.system("hspec96")
    os.system("hpulse96 -D -i > file96")
    os.system("f96tosac -B file96")

    # Filter and save the synthetic Green's functions
    greens = ("ZDD","RDD","ZDS","RDS","TDS","ZSS","RSS","TSS","ZEX","REX")

    for index,row in station_df.iterrows():      
        for j,grn in enumerate(greens):
            sacin = "B%03d%02d%s.sac"%(index+1,j+1,grn)
            sacout = "%s/%s.%s.%s.%.4f"%(green_dir,row.network,row.station,row.location,depth)
            tmp = read(sacin,format="SAC")
            tmp.filter('bandpass',freqmin=freqmin,freqmax=freqmax,corners=corners,zerophase=True)
            filename = "%s.%s"%(sacout,grn) 
            tmp.write(filename,format="SAC") # overwrite
        print("Output synthetics %s..." % (filename[:-3]) )

# Uncomment to remove unfiltered synthetic SAC files
os.system("rm B*.sac") # remove the unfiltered SAC files
print('Done!')

Output synthetics 41423391/gil7/BK.MTOS.00.10.0000....
Output synthetics 41423391/gil7/BK.FARB.00.10.0000....
Output synthetics 41423391/gil7/BK.MNRC.00.10.0000....
Output synthetics 41423391/gil7/BK.WELL.00.10.0000....
Output synthetics 41423391/gil7/BK.FORD.00.10.0000....
Output synthetics 41423391/gil7/BK.SAO.00.10.0000....
Output synthetics 41423391/gil7/BK.MTOS.00.20.0000....
Output synthetics 41423391/gil7/BK.FARB.00.20.0000....
Output synthetics 41423391/gil7/BK.MNRC.00.20.0000....
Output synthetics 41423391/gil7/BK.WELL.00.20.0000....
Output synthetics 41423391/gil7/BK.FORD.00.20.0000....
Output synthetics 41423391/gil7/BK.SAO.00.20.0000....
Output synthetics 41423391/gil7/BK.MTOS.00.30.0000....
Output synthetics 41423391/gil7/BK.FARB.00.30.0000....
Output synthetics 41423391/gil7/BK.MNRC.00.30.0000....
Output synthetics 41423391/gil7/BK.WELL.00.30.0000....
Output synthetics 41423391/gil7/BK.FORD.00.30.0000....
Output synthetics 41423391/gil7/BK.SAO.00.30.0000....
Output synthe

In [7]:
# 3.3 Create input file for MTtime
#   Now that we have prepared the data and synthetics for inversion.
#   we can create the input file for tdmtpy.
#   I will go over the input file format in the next notebook.


# Create headers
headers = dict(datetime=df["origin"][0],
               longitude=df["lon"][0],
               latitude=df["lat"][0],
               depth=",".join([ "%.4f"%d for d in depths]),
               path_to_data=event_dir,
               path_to_green=green_dir,
               green="herrmann",
               components="ZRT",
               degree=5,
               weight="distance",
               plot=0,
               correlate=0,
              )

# Add station table
pd.options.display.float_format = "{:,.2f}".format
frame = {"station": station_df[["network","station","location"]].apply(lambda x: ".".join(x),axis=1)}
df_out = pd.DataFrame(frame)
df_out[["distance","azimuth"]] = station_df[["distance","azimuth"]]
df_out["ts"] = int(30)
df_out["npts"] = int(150)
df_out["dt"] = dt
df_out["used"] = 1
df_out[["longitude","latitude"]] = station_df[["longitude","latitude"]]
#print(df_out.to_string(index=False))

Save to file **mtinv.in**

In [12]:
# write
with open("mtinv.in","w") as f:
    for key, value in headers.items():
        f.write("{0:<15}{1}\n".format(key,value))
    f.write(df_out.to_string(index=False))

# Now we can start the next tutorial and take a look at the moment tensor inversion package `mttime`